In [2]:
import json
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
import time

In [3]:
def areas_list_builder():
    areas = requests.get('http://reference.craigslist.org/Areas') # Make a get request to retrieve the page
    soup_areas = BeautifulSoup(areas.content, 'html.parser') # Pass the page contents to beautiful soup for parsing
    areas_list = json.loads(str(soup_areas))
    
    areas_list_final = []

    for area in areas_list:
        area = area['ShortDescription'].lower().replace(" ","")
        area = area.replace(",","")
        areas_list_final.append(area)
    
    return areas_list, areas_list_final

In [4]:
def dataframe_builder():    
    #features lists
    date = []
    price = []
    greater_area = []
    post_id = []
    desc = []

    all_areas, all_areas_url = areas_list_builder()

    for index, area in enumerate(all_areas_url): 
        try:
            
            
            
            regex_id = re.compile("postid_(.*)")
            listings = soup.find_all('li', class_="result-row") 

            for listing in listings:
                datetime = listing.find('time')['datetime'] if listing.find('time')['datetime'] else np.nan
                result_price = listing.find('span', class_="result-price").text if listing.find('span', class_="result-price") else np.nan
                #result_hood = listing.find('span', class_="result-hood").text.strip().strip("()") if listing.find('span', class_="result-hood") else np.nan

                date.append(datetime)
                price.append(result_price)
                greater_area.append(all_areas[index])

                for item in listing.find_all('a', {'id' : regex_id}):
                    result_id = item.attrs['id'] if item.attrs['id'] else np.nan 
                    result_desc = item.text if item else np.nan

                    post_id.append(result_id)
                    desc.append(result_desc)
        except:
            pass

        time.sleep(10)
            
    df = pd.DataFrame({
    'date': date,
    'post_id': post_id,
    'price': price,
    'description': desc,
    'sub_area': greater_area
    })
    
    return df

In [5]:
data = dataframe_builder()
print(data.head())
print(data.info())
print(data.shape)

               date            post_id price  \
0  2021-03-26 14:00  postid_7286348572  $725   
1  2021-03-26 13:59  postid_7295902767  $725   
2  2021-03-26 13:59  postid_7295904671  $725   
3  2021-03-26 13:44  postid_7297470882  $650   
4  2021-03-26 13:24  postid_7297386317  $800   

                           description  \
0     Playstation 5 (PS5) Disc Edition   
1     Playstation 5 (PS5) Disc Edition   
2  Sony Playstation 5 PS5 Disk Console   
3           Playstation 5 Disc Edition   
4                    PlayStation 5 PS5   

                                            sub_area  
0  {'Abbreviation': 'sfo', 'AreaID': 1, 'Country'...  
1  {'Abbreviation': 'sfo', 'AreaID': 1, 'Country'...  
2  {'Abbreviation': 'sfo', 'AreaID': 1, 'Country'...  
3  {'Abbreviation': 'sfo', 'AreaID': 1, 'Country'...  
4  {'Abbreviation': 'sfo', 'AreaID': 1, 'Country'...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8419 entries, 0 to 8418
Data columns (total 5 columns):
date           8419 no

In [7]:
#data.to_csv('craigslist_3.26.21.csv', index = True)